In [ ]:
from PIL import, Image
import numpy as np
import time
import random
from mss import mss
import sys
import pynput.keyboard as kb
import pynput.mouse as m
import os
from collections import defaultdict
from datetime import datetime

from enum import Enum, auto
import imagehash

import os

from os import listdir
from os.path import isfile, join

verbose = len(sys.argv) > 1 and sys.argv[1] == "-v"

In [ ]:
def load_masks(paths):
    masks = []
    for path in paths:
        img = Image.open(path)
        arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
        masks.append(arr)
    return masks

def debug(*args, **kwargs):
    if verbose:
        print(*args, **kwargs)


def same_images(A, B, delta = 5):
    dist = np.mean(np.sqrt(np.sum(np.square(A - B), axis = 2)))
    return dist < delta

def image_dist(A, B):
    dist = np.mean(np.sqrt(np.sum(np.square(A - B), axis = 2)))
    return dist

def same_colors(a, b, delta = 10):
    dist = np.linalg.norm(a-b)
    return dist < delta

def color_dist(a, b):
    return np.linalg.norm(a-b)


def _capture_portion_windows(x0, y0, w0, h0):
    x = x0/2 + WIN_OFFSET[0]
    y = y0/2 + WIN_OFFSET[1]
    w = w0/2 + 2
    h = h0/2 + 2

    with mss() as sct:
        monitor = {'left': int(x), 'top': int(y), 'width': int(w), 'height': int(h)}
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    
    img = img.repeat(2,axis=0).repeat(2,axis=1)
    img = img[x0%2:, x0%2:, :]
    img = img[:w0, :h0, :]
    return img

def _capture_portion_mac(x, y, w, h):
    x = x + MAC_OFFSET[0]
    y = y + MAC_OFFSET[1]

    with mss() as sct:
        monitor = {'left': int(x/2), 'top': int(y/2), 'width': int(w/2 + 1), 'height': int(h/2 + 1)}
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]

    img = img[x%2:, y%2:, :]
    img = img[:w, :h, :]
    return img

def capture_portion(x, y, w, h):
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    if PLATFORM == "windows":
        return _capture_portion_windows(x, y, w, h)
    else:
        return _capture_portion_mac(x, y, w, h)

def capture_screen():
    w, h = MAC_DIMS
    return capture_portion(0, 0, w, h)

def capture_whole_screen():
    with mss() as sct:
        monitor = sct.monitors[1]
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    
    if PLATFORM == "windows":
        img = img.repeat(2,axis=0).repeat(2,axis=1)

    return img

def capture_pixel(x, y):
    return capture_portion(x, y, 1, 1)[0,0,:]

def show_image(arr, title = None):
    img = Image.fromarray(arr.swapaxes(0, 1).astype(np.uint8), "RGB")
    if title is not None:
        img.show(title = title)
    else:
        img.show()

def save_image(arr, fname=None):
    if fname is None:
        fname = "saved/{}.png".format(time.time())
    img = Image.fromarray(arr.swapaxes(0, 1).astype(np.uint8), "RGB")
    img.save(fname)

def load_image(fname):
    img = Image.open(fname)
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    return arr

def save_game_capture(fname=None):
    img = capture_screen()
    if fname is None:
        fname = datetime.now().strftime(f'saved/%Y-%m-%d %H.%M.%S.png')
    save_image(img, fname)


def calibrate_offset():
    screen = capture_whole_screen()

    screen_top = -1
    screen_bot = -1
    screen_right = -1
    screen_left = -1

    for x in range(0, screen.shape[0], MAC_DIMS[0] + 8):
        stride = 0
        for y in range(screen.shape[1]):
            if same_colors(screen[x][y], [0, 0, 0], 0.05):
                stride += 1
            else:
                if stride == MAC_DIMS[1] + 8:
                    screen_top = y - stride + 4
                    screen_bot = y - 4
                elif stride == 4:
                    if screen_top == -1:
                        screen_top = y
                    elif screen_top == y - MAC_DIMS[1] + 4:
                        screen_bot = y - 4
                stride = 0

    for y in range(0, screen.shape[1], MAC_DIMS[1] + 8):
        stride = 0
        for x in range(screen.shape[0]):
            if same_colors(screen[x][y], [0, 0, 0], 0.05):
                stride += 1
            else:
                if stride == MAC_DIMS[0] + 8:
                    screen_left = x - stride + 4
                    screen_right = x - 4
                elif stride == 4:
                    if screen_left == -1:
                        screen_left = x
                    elif screen_left == x - MAC_DIMS[0] + 4:
                        screen_right = x - 4
                stride = 0
    
    debug("screen bounds: ({}, {}) to ({}, {})".format(screen_left, screen_top, screen_right, screen_bot))

    if screen_top != -1 and screen_bot != -1 and screen_right != -1 and screen_left != -1:
        if PLATFORM == "macOS":
            global MAC_OFFSET
            MAC_OFFSET = (screen_left, screen_top)
        else:
            global WIN_OFFSET
            WIN_OFFSET = (int(screen_left)/2, int(screen_top)/2)
    else:
        debug("trouble finding all edges of screen")

def is_game_over_screen():
    pixel = capture_pixel(680, 380)     # piggy bank
    return same_colors(pixel, [246, 196, 195], 10)

def is_arcade_screen():
    pixel = capture_pixel(170, 852)     # magenta arcade cabinet
    return same_colors(pixel, [143, 60, 148], 10)


def is_easy_mode():
    level_img = capture_portion(454, 23, 45, 45)
    return same_images(level_img, level_masks[30])

def get_best_mask(target, masks):
    best_dist = -1
    best_idx = 0
    for i, img in enumerate(masks):
        dist = np.mean(np.sqrt(np.sum(np.square(target - img), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_idx = i
    return best_idx


def get_level():
    if is_easy_mode():
        level_img = capture_portion(615, 23, 45, 45)
    else:
        level_img = capture_portion(454, 23, 45, 45)
    curr_level = str(get_best_mask(level_img, level_masks[:-1]) + 1)
    return curr_level

def get_time():
    if is_easy_mode():
        return -1
    time_img = capture_portion(751, 30, 68, 30)
    digit_minutes = get_best_mask(time_img[:20, :30], time_masks)
    digit_tens = get_best_mask(time_img[28:28+20, :30], time_masks)
    digit_ones = get_best_mask(time_img[48:48+20, :30], time_masks)
    return digit_minutes*60 + digit_tens*10 + digit_ones


def get_digit(i, j):
    x0, y0 = (687,164)
    sx, sy = (32, 44)
    w, h = (30, 30)

    x = x0 + sx * j
    y = y0 + sy * i

    digit_img = capture_portion(x, y, w, h)

    idx = get_best_mask(digit_img, digit_masks)
    if idx == 10:
        digit = '-'
    elif idx == 11:
        digit = ''
    else:
        digit = str(idx)

    return digit


def press_key(key):
    keyboard.press(key)
    time.sleep(random.uniform(0.1, 0.15))
    keyboard.release(key)
    time.sleep(random.uniform(0.1, 0.15))
    
def click_mouse(x, y):
    if PLATFORM == "macOS":
        x = (x + MAC_OFFSET[0])/2
        y = (y + MAC_OFFSET[1])/2
    else:
        x = x + WIN_OFFSET[0]
        y = y + WIN_OFFSET[1]
    mouse.position = (x, y)
    time.sleep(random.uniform(0.2, 0.5))
    mouse.press(m.Button.left)
    time.sleep(random.uniform(0.1, 0.15))
    mouse.release(m.Button.left)
    time.sleep(random.uniform(0.1, 0.15))

def highlight_color(img, target):
    img = img.copy()
    img[(img != target).any(axis=2)] //= 10
    return img


def highlight_colors(img, targets):
    img = img.copy()
    mask = np.zeros(img.shape)
    for target in targets:
        mask[(img == target).all(axis=2)] = 1
    return img * mask

def interval(line):
    interval = []
    length = 0
    for i, pix in enumerate(line):
        length += 1
        if i == len(line) - 1 or (line[i+1] != line[i]).any():
            interval.append(pix)
            length = 0
    return interval

def locate_cards():
    res = capture_screen()
    # colors, counts = np.unique(res[:, 450, :], axis=0, return_counts=True)
    # for color, k in zip(colors, counts):
    #     if k > 2:
    #         print(color, k)
    #         show_image(highlight_color(res, target=color), title=color)

    mask = highlight_colors(res, targets=[CARD_BORDER_YELLOW, CARD_STRIPE_D_BLUE, CARD_BG_L_BLUE])
    top_blue = 900
    bot_blue = 0
    rt_blue = 0
    left_blue = 1200

    row = mask[:900, 600, :]
    

    for y, row in enumerate(mask):
        for x, pix in enumerate(row):
            if (pix == CARD_BORDER_D_BLUE).all():
                top_blue = min(top_blue, y)
                bot_blue = max(bot_blue, y)
                rt_blue = max(rt_blue, x)
                left_blue = min(left_blue, x)
    print(left_blue, top_blue, rt_blue - left_blue, bot_blue - top_blue)
   

    for y, row in enumerate(mask):
        for x, pix in enumerate(row):
            if (pix == CARD_BORDER_D_BLUE).all():
                top_blue = min(top_blue, y)
                bot_blue = max(bot_blue, y)
                rt_blue = max(rt_blue, x)
                left_blue = min(left_blue, x)
    print(left_blue, top_blue, rt_blue - left_blue, bot_blue - top_blue)
    # show_image(capture_portion(left_blue, top_blue, left_blue - rt_blue, bot_blue - top_blue))

    # row = mask[:, 450, :]
    # print(interval(row))

    # pattern = [..., blue, yellow, .... yellow, blue, ..., blue, ...]

class Tile(Enum):
    UNKOWN = auto(),
    ANT_UP = auto(),
    ANT_DOWN = auto(),
    ANT_RIGHT = auto(),
    ANT_LEFT = auto(),
    ANT_TR = auto(),
    ANT_TL = auto(),
    ANT_BR = auto(),
    ANT_BL = auto(),
    BLANK_LBLUE = auto(),
    BLANK_WHITE = auto(),
    BLANK_DBLUE = auto(),
    FOOD_TOMATO = auto(),
    FOOD_CHICKEN = auto(),
    FOOD_BANANA = auto(),
    FOOD_HOTDOG = auto(),
    FOOD_PEAR = auto(),
    FOOD_SANDWICH = auto(),
    FOOD_SODA = auto(),
    FOOD_YOGURT = auto(),
    FOOD_STRAWBERRY = auto(),
    FOOD_LOLLIPOP = auto(),
    FOOD_PIZZA = auto(),
    FOOD_CAKE = auto()

tile_indicators = {
    Tile.ANT_UP: [[27, 13], [151, 151, 151]],
    Tile.ANT_DOWN: [[25, 41], [151, 151, 151]],
    Tile.ANT_RIGHT: [[41, 28], [151, 151, 151]],
    Tile.ANT_LEFT: [[11, 25], [151, 151, 151]],
    Tile.ANT_TR: [[37, 17], [151, 151, 151]],
    Tile.ANT_TL: [[42, 38], [145, 144, 144]],
    Tile.ANT_BR: [[36, 37], [151, 151, 151]],
    Tile.ANT_BL: [[15, 36], [151, 151, 151]],
    Tile.FOOD_TOMATO: [[16, 9], [64, 100, 57]],
    Tile.FOOD_CHICKEN: [[12, 11], [146, 104, 60]],
    Tile.FOOD_BANANA: [[30, 39], [119, 61, 20]],
    Tile.FOOD_HOTDOG: [[27, 35], [198, 167, 101]],
    Tile.FOOD_PEAR: [[26, 22], [191, 238, 76]],
    Tile.FOOD_SANDWICH: [[26, 26], [236, 216, 175]],
    Tile.FOOD_SODA: [[22, 8], [0, 0, 0]],
    Tile.FOOD_YOGURT: [[29, 19], [190, 158, 198]],
    Tile.FOOD_STRAWBERRY: [[19, 9], [103, 200, 59]],
    Tile.FOOD_LOLLIPOP: [[26, 3], [231, 164, 93]],
    Tile.FOOD_PIZZA: [[13, 15], [197, 154, 71]],
    Tile.FOOD_CAKE: [[29, 32], [235, 74, 151]]
}

tile_ids = {
    Tile.ANT_UP: ["ant_up.png"],
    Tile.ANT_DOWN: ["ant_down.png"],
    Tile.ANT_RIGHT: ["ant_right.png"],
    Tile.ANT_LEFT: ["ant_left.png"],
    Tile.ANT_TR: ["ant_top_right.png"],
    Tile.ANT_TL: ["ant_top_left.png"],
    Tile.ANT_BR: ["ant_bottom_right.png"],
    Tile.ANT_BL: ["ant_bottom_left.png"],
    Tile.BLANK_LBLUE: ["blank_lblue.png"],
    Tile.BLANK_DBLUE: ["blank_dblue.png"],
    Tile.BLANK_WHITE: ["blank_white.png"],
    Tile.FOOD_TOMATO: ["tomato.png"],
    Tile.FOOD_CHICKEN: ["chicken.png"],
    Tile.FOOD_BANANA: ["banana_white.png", "banana_lblue.png", "banana_dblue.png"],
    Tile.FOOD_HOTDOG: ["hotdog.png"],
    Tile.FOOD_PEAR: ["pear.png"],
    Tile.FOOD_SANDWICH: ["sandwich.png"],
    Tile.FOOD_SODA: ["soda.png"],
    Tile.FOOD_YOGURT: ["yogurt.png"],
    Tile.FOOD_STRAWBERRY: ["strawberry.png"],
    Tile.FOOD_LOLLIPOP: ["lollipop_lblue_right.png", "lollipop_lblue_left.png","lollipop_dblue_right.png", "lollipop_dblue_left.png","lollipop_white_right.png", "lollipop_white_left.png"],
    Tile.FOOD_PIZZA: ["pizza.png"],
    Tile.FOOD_CAKE: ["cake.png"]
}

def get_best_tile_info(tile_ids, img):
    best_dist = -1
    best_type = Tile.UNKOWN
    best_file = None
    best_mask = None
    for tile_type, masks in tile_ids.items():
        for filename, mask in masks.items():
            dist = np.mean(np.sqrt(np.sum(np.square(img - mask), axis = 2)))
            if best_dist == -1 or dist < best_dist:
                best_dist = dist
                best_type = tile_type
                best_file = filename
                best_mask = mask
    return best_type, best_file, best_dist, best_mask

def get_tile_type(tile_center):
    best_type, best_file, best_dist, best_mask = get_best_tile_info(tile_ids, tile_center)
    if best_dist > MAX_DIST:
        # debug("UNKNOWN:", best_type, best_file, best_dist)
        best_type = Tile.UNKOWN
    return best_type

tile_char = {
    Tile.ANT_UP: "↑",
    Tile.ANT_DOWN: "↓",
    Tile.ANT_RIGHT: "→",
    Tile.ANT_LEFT: "←",
    Tile.ANT_TR: "↗",
    Tile.ANT_TL: "↖",
    Tile.ANT_BR: "↘",
    Tile.ANT_BL: "↙",
    Tile.FOOD_TOMATO: "🍅",
    Tile.FOOD_CHICKEN: "🍗",
    Tile.FOOD_BANANA: "🍌",
    Tile.FOOD_HOTDOG: "🌭",
    Tile.FOOD_PEAR: "🍐",
    Tile.FOOD_SANDWICH: "🥪",
    Tile.FOOD_SODA: "🥤",
    Tile.FOOD_YOGURT: "🥣",
    Tile.FOOD_STRAWBERRY: "🍓",
    Tile.FOOD_LOLLIPOP: "🍭",
    Tile.FOOD_PIZZA: "🍕",
    Tile.FOOD_CAKE: "🍰",
    Tile.BLANK_LBLUE: "▒",
    Tile.BLANK_WHITE: "░",
    Tile.BLANK_DBLUE: "▓",
    Tile.UNKOWN: "❓"
}


def remove_background(bg, img):
    img.copy()
    img[(img == bg).all(axis=2)] *= 0
    return img


def get_best_mask_dict(target, masks):
    best_dist = -1
    best_idx = ""
    for file, img in masks.items():
        dist = np.mean(np.sqrt(np.sum(np.square(target - img), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_idx = file
    return best_idx, best_dist

def is_food(tile_type):
    return tile_type.name[:4] == "FOOD"

def is_ant(tile_type):
    return tile_type.name[:3] == "ANT"

def is_blank(tile_type):
    return tile_type.name[:5] == "BLANK"

def is_unknown(tile_type):
    return tile_type == Tile.UNKNOWN

def capture_tile(i, j):
    x = i*cell_dims[0]
    y = j*cell_dims[1]
    w = cell_dims[0]
    h = cell_dims[1]
    return capture_portion(x, y, w, h)

def capture_tile_center(i, j, size = 8):
    x = blanket_offset[0] + i*cell_dims[0] + (52 - size) // 2
    y = blanket_offset[1] + j*cell_dims[1] + (52 - size) // 2
    return capture_portion(x, y, size, size)

hamiltonian_path = [
    (np.array([13, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([12, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([11, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([10, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([9, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([8, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([7, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([6, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([5, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([4, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([3, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([2, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([1, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([0, 0]), [kb.Key.right, kb.Key.right]),
    (np.array([25, 0]), [kb.Key.down, kb.Key.down]),
    (np.array([25, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([24, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([23, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([22, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([21, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([20, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([19, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([18, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([17, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([16, 1]), [kb.Key.left, kb.Key.down]),
    (np.array([15, 17]), [kb.Key.left, kb.Key.up]),
    (np.array([14, 1]), [kb.Key.left, kb.Key.down])
]

step = 0

# Excellent for medium difficulty
def quick_u_turn_medium(keys):
    keyboard.press(keys[0])
    time.sleep(0.05)
    keyboard.press(keys[1])
    time.sleep(0.05)
    keyboard.release(keys[0])
    time.sleep(0.05)
    keyboard.release(keys[1])

def quick_u_turn_easy(keys):
    keyboard.press(keys[0])
    time.sleep(0.10)
    keyboard.press(keys[1])
    time.sleep(0.05)
    keyboard.release(keys[0])
    time.sleep(0.05)
    keyboard.release(keys[1])

def quick_u_turn_hard(keys):
    keyboard.press(keys[0])
    time.sleep(0.05)
    keyboard.press(keys[1])
    time.sleep(0.02)
    keyboard.release(keys[0])
    time.sleep(0.05)
    keyboard.release(keys[1])



def quick_is_ant(i, j):
    tile_center = capture_tile_center(i, j, size = MASK_SIZE)
    dist = image_dist(tile_center, avg_ant_mask)
    return dist < 70

dir_offset = {
    kb.Key.right: np.array([1, 0]),
    kb.Key.left: np.array([-1, 0]),
    kb.Key.up: np.array([0, -1]),
    kb.Key.down: np.array([0, 1])
}

def get_cell_in_direction(pos, direction):
    return pos + dir_offset[direction]


In [ ]:
# macOS, use native values rgb picker
# macOS, Safari: 16" MBP 2019, resolution scaled more space, resolution (4096x2560)
MAC_OFFSET = (878, 546)             # Safari (in top left)
# MAC_OFFSET = (1449, 797)          # Desktop app (centered)
MAC_DIMS = (1600, 1200)
# windows: Microsoft edge, win+<-, my desktop, monitor (1920x1080)
WIN_OFFSET = (389, 230)
WIN_DIMS = (800, 600)

if os.name == "nt":
    PLATFORM = "windows"
elif os.name == "posix":
    PLATFORM = "macOS"



keyboard = kb.Controller()
mouse = m.Controller()

game_start = None

from skimage import io
# calibrate_offset()

# levels = [[2, 2], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]]

# l = 0
# level = l + 1

# board_pos = [[256, 161], [83, 161], [255, 157]]

# board_height = 692
# card_dims = card_height = board_height / levels[l][1]
# board_width = int(card_dims * levels[l][0])
# board_x = board_pos[l][0]
# board_y = board_pos[l][1]


# board_pic = capture_portion(board_x, board_y, board_width, board_height)

# show_image(capture_screen())
# locate_cards()


grid_dims = np.array([26, 18])
cell_dims = np.array([52, 52])
blanket_offset = np.array([125, 136])
blanket_dims = np.array([1352, 936])

# res = capture_screen()
# save_image(res)

# for tile_type, files in tile_ids.items():
#     imgs = {}
#     for file in files:
#         imgs[file] = load_image(f"unique_tiles/best_choices/{file}")
#     tile_ids[tile_type] = imgs

MAX_DIST = 100

# background = load_image("level_bg.png")
# significant = load_image("saved/2021-07-11 14.57.11.png")

# show_image(remove_background(background, significant))

# game_start = datetime.now()

# tile_img = load_image("tiles/food_strawberry_lblue.png")
# print((np.diagonal(tile_img, axis1=0, axis2=1).T[2:-2] == [127, 202, 250]).all())

# goal = load_image()

# i = 0
# while True:
#     i += 1
#     tile = capture_portion(801, 1020, 52, 52)
#     print(same_images(tile, ))
#     print(i)
# show_image(tile)



# blanket = capture_portion(blanket_offset[0], blanket_offset[1],blanket_dims[0],blanket_dims[1])
# show_image(blanket)

# grid_tiles = [[0 for j in range(grid_dims[1])] for i in range(grid_dims[0])]
# for i in range(grid_dims[0]):
#     for j in range(grid_dims[1]):
#         x = i*cell_dims[0]
#         y = j*cell_dims[1]
#         w = cell_dims[0]
#         h = cell_dims[1]
#         tile_img = blanket[x:x+w, y:y+h, :]

#         grid_tiles[i][j] = get_tile_type(tile_img)

# for row in np.array(grid_tiles).T.tolist():
#     print(' '.join([tile_char[x] for x in row]))


# show_image(blanket)


# onlyfiles = [f[:-4] for f in listdir("unique_tiles") if isfile(join("unique_tiles", f))]
# print(onlyfiles)

# distances = {}

# for folder in os.walk("unique_tiles"):
#     dir_path, _, filenames = folder
#     if dir_path == "unique_tiles":
#         continue
#     saved = set()
#     tile_images = {}
#     tiles = []
#     for filename in filenames:
#         if filename[-4:] != ".png":
#             continue
#         tile_mid = load_image(f"{dir_path}/{filename}")
#         tile_images[filename] = tile_mid
#         tiles.append(tile_mid)
    
#     distances[f"{dir_path}"] = {}

#     for i, tile_1 in enumerate(list(tile_images.keys())):
#         tile_img_1 = tile_images[tile_1]
#         max_dist = 0
#         for j, tile_2 in enumerate(list(tile_images.keys())):
#             if i == j:
#                 continue
#             tile_img_2 = tile_images[tile_2]
#             distance = image_dist(tile_img_1, tile_img_2)
#             if distance > max_dist:
#                 max_dist = distance
#         distances[f"{dir_path}"][tile_1] = max_dist
    
#     avg_tile = np.mean(np.array(tiles), axis=0)
#     save_image(avg_tile, f"{dir_path}/average.png")

# best_files = {}
# for folder in distances:
#     min_dist = 1000000000000
#     best_file = ""
#     for file in distances[folder]:
#         dist = distances[folder][file]
#         if dist < min_dist:
#             min_dist = dist
#             best_file = file
#     best_files[folder] = {best_file: min_dist}

# output = []
# for folder, item in best_files.items():
#     if folder == "unique_tiles/best_choices":
#         continue
#     filename, dist = list(item.items())[0]
#     save_image(load_image(f"{folder}/{filename}"), f"unique_tiles/best_choices/{folder.split('/')[1]}.png")
#     output.append([folder, filename, dist])
#     print(filename)

# output = sorted(output, key = lambda l:l[2], reverse=True)

# print('\n'.join(["\t".join([str(r) for r in row]) for row in output]))

# tile_ids = {}
# for file in os.listdir("unique_tiles/best_choices"):
#     if file[-4:] != ".png":
#         continue
#     tile_ids[file] = load_image(f"unique_tiles/best_choices/{file}")


# max_dist = 0
# for folder in os.walk("unique_tiles"):
#     dir_path, _, filenames = folder
#     if dir_path == "unique_tiles" or dir_path == "unique_tiles/best_choices":
#         continue
#     for filename in filenames:
#         if filename[-4:] != ".png":
#             continue
#         tile = load_image(f"{dir_path}/{filename}")
#         best, dist = get_best_mask_dict(tile, tile_ids)
#         if (folder[0].split('/')[1] != best[:-4]):
#             print(f"{dir_path}/{filename}", best, dist, sep = '\t')
#         if dist > max_dist:
#             max_dist = dist
# print(max_dist)
    # save_image(avg_tile, f"{dir_path}/average.png")

# k = 0
# while True:
#     blanket = capture_portion(blanket_offset[0], blanket_offset[1],blanket_dims[0],blanket_dims[1])
#     k += 1
#     grid_tiles = [[0 for j in range(grid_dims[1])] for i in range(grid_dims[0])]
#     for i in range(grid_dims[0]):
#         for j in range(grid_dims[1]):
#             x = i*cell_dims[0]
#             y = j*cell_dims[1]
#             w = cell_dims[0]
#             h = cell_dims[1]
#             tile_img = blanket[x:x+w, y:y+h, :]
#             part_to_hash = tile_img[22:30, 22:30, :]
#             # name = imagehash.average_hash(Image.fromarray(part_to_hash.swapaxes(0, 1).astype(np.uint8), "RGB"))
#             # name = imagehash.colorhash(Image.fromarray(part_to_hash.swapaxes(0, 1).astype(np.uint8), "RGB"))
#             name = hash(str(part_to_hash))
#             if name not in saved:
#                 saved.add(name)
#                 # name = f"unique_tiles/{name}.png"
#                 name = f"numpy_tiles/{name}.png"
#                 # name = f"color_hash/{name}.png"
#                 save_image(part_to_hash, name)
#                 print(f"saved {name}")
#     time.sleep(3)
#     for i in range(10):
#         print(f"{i}{'-'*i}")
#         time.sleep(0.01)
#     # grid_tiles[i][j] = get_tile_type(tile_img)

avg_ant_mask = load_image("unique_tiles/best_choices/ant.png")[3, 3, :]

while True:
    target_tile, directions = hamiltonian_path[step % len(hamiltonian_path)]
    pos = blanket_offset + 25 + target_tile*cell_dims 
    next_cell = get_cell_in_direction(target_tile, directions[0])
    next_pos = blanket_offset + 25 + next_cell*cell_dims 
    while True:
        dist = color_dist(capture_pixel(pos[0], pos[1]), avg_ant_mask)
        if dist < 60: # TODO: this will prob break when close to done
            next_center = capture_pixel(next_pos[0], next_pos[1])
            dist_2 = color_dist(next_center, avg_ant_mask)
            if not (dist_2 < 60):
                quick_u_turn_medium(directions)
                step += 1
                break

# test = load_image("unique_tiles/best_choices/lollipop_dblue_right.png")[3, 3, :]
# print(color_dist(avg_ant_mask, test))

# grid_tiles = [[0 for j in range(grid_dims[1])] for i in range(grid_dims[0])]
# for i in range(grid_dims[0]):
#     for j in range(grid_dims[1]):
#         tile_center = capture_tile_center(i, j)
#         grid_tiles[i][j] = get_tile_type(tile_center)

# for row in np.array(grid_tiles).T.tolist():
#     print(' '.join([tile_char[x] for x in row]))